# A single multi-band + directivity example analysis

In [ ]:
import masp as srs
import numpy as np
import soundfile as sf
from IPython.display import Audio
import scipy
import copy
import pandas as pd
import os
from os.path import join as pjoin
from multiprocessing import Pool
import matplotlib.pyplot as plt
import mat73
import tqdm
import librosa as lsa
import scipy.signal as sig

In [ ]:
'''  MASP CONVENTIONS

ROOM CONVENTION
              _____    
             |     |   
             |     |   
           x ^     |   
             |     |   
             |     |  
             o---->    
                  y

    headOrient -> clockwise rotation (towards room origin) [azimuth [-180,180], elevation [-90, 90]] 
    sourceOrient -> clockwise rotation (towards room origin) [azimuth [-180,180], elevation [-90, 90]]
    
    ECHOGRAM CONVENTION
                ^x
              __|__    
             |  |  |   
             |  |  |   
          y<----o  |   
             |     |   
             |     |   
             |_____| 
''';

In [ ]:
def load_speechdirectivity(path, plot):
    dirdata = scipy.io.loadmat(path)['azel_dir']
    d = {}
    bands = np.array(['100Hz', '125Hz', '160Hz', '200Hz', '250Hz', '315Hz', '400Hz', '500Hz', '630Hz', '800Hz', '1000Hz', '1250Hz', '1600Hz', '2000Hz', '2500Hz', '3150Hz', '4000Hz', '5000Hz', '6300Hz', '8000Hz', '10000Hz'])
    for i, band in enumerate(bands):
        d[band] = dirdata[i]
    az_axis = np.linspace(-180, 175, 72)
    el_axis = np.linspace(-90, 85, 36)
    d['az_axis'] = az_axis
    d['el_axis'] = el_axis
    if plot:
        plt.figure(figsize=(12,20))
        for i, band in enumerate(bands):
            plt.subplot(7,3,i+1)
            plt.imshow(d[band], cmap='jet')
            plt.yticks(range(len(d['el_axis']))[::8], [int(x) for x in d['el_axis'][::8]], fontsize=7)
            plt.ylabel('elevation')
            plt.xticks(range(len(d['az_axis']))[::8], [int(x) for x in d['az_axis'][::8]], rotation=90, fontsize=7)
            plt.xlabel('azimuth')
            plt.title(band)
            plt.clim(-20,0)
        cbar_ax = plt.gcf().add_axes([0.92, 0.15, 0.02, 0.72])  # [left, bottom, width, height]
        cbar = plt.colorbar(cax=cbar_ax)
        plt.savefig('speech_directivity.pdf')
    return d

def get_6band_rt60_vector():
    np.random.seed() #we randomize so multiprocessing doesn't yield same RT60s
    alphas = np.array([1.7196874268124676,
                         1.6152228672267106,
                         1.9318203836226113,
                         2.55718115999814,
                         4.176814897493042,
                         2.4892656080814346])
    betas = np.array([0.38685390302225775,
                         0.24453641709737417,
                         0.14321372785643122,
                         0.10453218827453133,
                         0.08678871224845529,
                         0.18290733668646034])
    sim_rt60Fs = []
    for i in range(len(alphas)):
        sim_rt60Fs.append(np.random.gamma(alphas[i], betas[i], 1))
    return np.array(sim_rt60Fs).squeeze()

def crop_echogram(anechoic_echogram):
    nSrc = anechoic_echogram.shape[0]
    nRec = anechoic_echogram.shape[1]
    nBands = anechoic_echogram.shape[2]
    # Returns the "anechoic" version of an echogram
    # Should keep the receiver directivy
    for src in range(nSrc):
        for rec in range(nRec):
            for band in range(nBands):
                anechoic_echogram[src, rec, band].time = anechoic_echogram[src, rec, band].time[:2]
                anechoic_echogram[src, rec, band].coords = anechoic_echogram[src, rec, band].coords[:2, :]
                anechoic_echogram[src, rec, band].value = anechoic_echogram[src, rec, band].value[:2,:]
                anechoic_echogram[src, rec, band].order = anechoic_echogram[src, rec, band].order[:2,:]
    return anechoic_echogram
    
def head_2_ku_ears(head_pos,head_orient):
# based on head pos and orientation, compute coordinates of ears
    ear_distance_ku100=0.0875
    theta = (head_orient[0]) * np.pi / 180
    R_ear = [head_pos[0] - ear_distance_ku100 * np.sin(theta),
              head_pos[1] + ear_distance_ku100 * np.cos(theta), 
              head_pos[2]]
    L_ear = [head_pos[0] + ear_distance_ku100 * np.sin(theta),
              head_pos[1] - ear_distance_ku100 * np.cos(theta), 
              head_pos[2]]
    return [L_ear,R_ear]
    
def plot_scene(room_dims,head_pos,head_orient,l_mic_pos,l_src_pos, perspective="xy"):
#   function to plot the designed scene
#   room_dims - dimensions of the room [x,y,z]
#   head_pos - head position [x,y,z]
#   head_orient - [az,el]
#   l_src_pos - list of source positions [[x,y,z],...,[x,y,z]]
#   ref_pos - reflection to plot position
#   perspective - which two dimensions to show 
    if perspective=="xy":
        dim1=1
        dim2=0
    elif perspective=="yz":
        dim1=2
        dim2=1
    elif perspective=="xz":
        dim1=2
        dim2=0
    fig = plt.figure()
    ax = fig.add_subplot()
    plt.xlim((0,room_dims[dim1]))
    plt.ylim((0,room_dims[dim2]))
    plt.axvline(head_pos[dim1], color='y') # horizontal lines
    plt.axhline(head_pos[dim2], color='y') # vertical lines
    plt.grid(True)
    # plot sources and receivers
    plt.plot(head_pos[dim1],head_pos[dim2], "o", ms=10, mew=2, color="black")
    # plot ears
    plt.plot(l_mic_pos[0][dim1],l_mic_pos[0][dim2], "o", ms=3, mew=2, color="blue")# left ear in blue
    plt.plot(l_mic_pos[1][dim1],l_mic_pos[1][dim2], "o", ms=3, mew=2, color="red")# right ear in red

    for i,src_pos in enumerate(l_src_pos):
        plt.plot(src_pos[dim1],src_pos[dim2], "o", ms=10, mew=2, color="red")
        plt.annotate(str(i), (src_pos[dim1],src_pos[dim2]))

    # plot head orientation if looking from above 
    if perspective=="xy":
        plt.plot(head_pos[dim1],head_pos[dim2], marker=(1, 1, -head_orient[0]), ms=20, mew=2,color="black")

    ax.set_aspect('equal', adjustable='box')
    
def add_azi(ang1, ang2):
    # ang1 and ang2 are defined from [-180 to 180]
    result = ang1 + ang2
    if result < -180:
        result += 360
    elif  result > 180:
        result -= 360   
    if result == 180:
        result = -180
    return result

def add_ele(ang1, ang2):
    # ang1 and ang2 are defined from [-180 to 180]
    result = ang1 + ang2
    if result < -90:
        result += 180
    elif  result > 90:
        result -= 180   
    if result == 90:
        result = -90
    return result
    
def apply_directivity(echograms, recip_echograms, sourceOrient, d, band_centerfreqs):
    # echograms: absorption echograms with shape [source, receiver, band]
    # recip_echograms: absorption reciprocal echograms with shape [receiver, source, band]
    # sourceOrient: clockwise source rotation in degrees, from -180 to 180 azimuth and -90 to 90 elevation, [azi, el]
    # d: average speech directivity dict, with d['100Hz'].shape == [36 (elevation),72(azimuth)]
    directivity_echograms = copy.deepcopy(echograms)
    for ns in range(nSrc): # for each source
        for nr in range(nRec): # for each receiver 
            for bd in range(directivity_echograms.shape[2]): # for each band
                band = str(int(band_centerfreqs[bd]))+'Hz'
                # pass reciprocal coordinates to azimuth elevation
                sph = srs.utils.cart2sph(recip_echograms[nr, ns, bd].coords)
                azi = 180 * sph[:,0] / np.pi
                ele = 180 * sph[:,1] / np.pi # rad to degrees
                #qazi = np.zeros_like(azi)
                azi_rot = np.zeros_like(azi)
                azi_idxs = np.zeros_like(azi).astype(int)
                for i in range(len(azi)):
                    azi_rot[i] = add_azi(azi[i], sourceOrient[0])
                    azi_idxs[i] = int(np.argmin(np.abs(d['az_axis'] - azi_rot[i])))
                    #qazi[i] = d['az_axis'][azi_idxs[i]]
                #qele = np.zeros_like(ele)
                ele_rot = np.zeros_like(ele)
                ele_idxs = np.zeros_like(ele).astype(int)
                for i, angle in enumerate(ele):
                    ele_rot[i] = add_azi(ele[i], sourceOrient[1])
                    ele_idxs[i] = int(np.argmin(np.abs(d['el_axis'] - ele_rot[i])))
                    #qele[i] = d['el_axis'][ele_idxs[i]]
                factors = np.zeros(len(directivity_echograms[ns,nr,bd].value))
                for r in range(len(factors)):
                    factors[r] = np.power(10, d[band][ele_idxs[r], azi_idxs[r]] / 20)
                factors = np.tile(factors, (directivity_echograms[ns,nr,bd].value.shape[1], 1)).T
                directivity_echograms[ns, nr, bd].value *= factors
    return directivity_echograms

In [ ]:
directivity_data = load_speechdirectivity(path=pjoin('directivity_parsing_matlab', 'azel_dir.mat'), plot=False)

#rt60s = get_6band_rt60_vector()
rt60s = np.array([0.5, 0.4, 0.3, 0.2, 0.1, 0.01])
band_centerfreqs = np.zeros((len(rt60s)))
band_centerfreqs[0] = 125
for nb in range(5):
    band_centerfreqs[nb+1] = 2 * band_centerfreqs[nb]

In [ ]:
rt60 = np.mean(rt60s)

In [ ]:
decoder_path = pjoin('decoders_ord10', 'Ku100_ALFE_Window_sinEQ_bimag.mat') #10th order BimagLS decoder del KU100 sin HA a 48kHz
decoder = mat73.loadmat(decoder_path)['hnm']
decoder = np.roll(decoder,500,axis=0)
maxlim = 2 # maximum reflection time in seconds. Stop simulating if it goes beyond that time.
ambi_order = 10 # ambisonics order

headC_x = 3.0  
headC_y = 3.0
headC_z = 1.0
headOrient_azi = 0.0
headOrient_ele = 0.0
headC = np.array([headC_x, headC_y, headC_z])
headOrient = np.array([headOrient_azi,headOrient_ele])
src = np.array([[5,	3, 1], [3, 3, 3], [1,3, 1], [3, 3 , 0.1]]) #speech speaker position following convention:

room = np.array([6, 6, 4]) #dimensions
fs_rir = 48000
fs_target = fs_rir

speech, fs_speech = lsa.load('ane_speech.wav', sr=fs_rir)

In [ ]:
#mic = np.array(head_2_ku_ears(headC,headOrient)) # we get BiMagLS mic points 
#mic = np.vstack((mic, headC)) # we add the head center microphone for MagLS decoders
mic = np.array([headC])
nRec = mic.shape[0]
nSrc = src.shape[0]

In [ ]:
abs_walls,_ = srs.find_abs_coeffs_from_rt(room, rt60s)

In [ ]:
limits = np.minimum(rt60s, maxlim)

In [ ]:
echograms  = srs.compute_echograms_sh(room, src, mic, abs_walls, limits, ambi_order, headOrient)
recip_echograms  = srs.compute_echograms_sh(room, mic, src, abs_walls, limits, ambi_order, headOrient)

In [ ]:
'''
# We build a dummy directivity pattern that only directs sound in the [-45, 45] azimuthal plane (all elevations)
d2 = directivity_data.copy()
newdir = np.ones_like(d2['125Hz'])
newdir *= -9E9
newdir[:,27:46] = 0
for band in np.array(['100Hz', '125Hz', '160Hz', '200Hz', '250Hz', '315Hz', '400Hz', '500Hz', '630Hz', '800Hz', '1000Hz', '1250Hz', '1600Hz', '2000Hz', '2500Hz', '3150Hz', '4000Hz', '5000Hz', '6300Hz', '8000Hz', '10000Hz']):
    d2[band] = newdir
''';
# We build a dummy directivity pattern that only directs sound in the [-45, 45] elevation plane (all azimuths)
d2 = directivity_data.copy()
newdir = np.ones_like(d2['125Hz'])
newdir *= -9E9
newdir[10:27, :] = 0
for band in np.array(['100Hz', '125Hz', '160Hz', '200Hz', '250Hz', '315Hz', '400Hz', '500Hz', '630Hz', '800Hz', '1000Hz', '1250Hz', '1600Hz', '2000Hz', '2500Hz', '3150Hz', '4000Hz', '5000Hz', '6300Hz', '8000Hz', '10000Hz']):
    d2[band] = newdir
    

In [ ]:
sourceOrient = np.array([0,-90])

In [ ]:
directivity_echograms = apply_directivity(echograms, recip_echograms, sourceOrient, d2, band_centerfreqs)

In [ ]:
plot_scene(room,mic[0],headOrient,head_2_ku_ears(mic[0], headOrient),src, perspective="xz")

In [ ]:
directivity_echograms[1,0,0].value[0,:]